<a href="https://colab.research.google.com/github/Abhinav6300219502/Tensor-flow-and-Pytorch/blob/main/Lab03_TensorFlow_vs_PyTorch_(4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 03: TensorFlow vs. PyTorch
- Train a model on MNIST in both TensorFlow and PyTorch, convert to TFLite and ONNX.  
- Use tf.GradientTape for Tensorflow custom training loop.



## TensorFlow Implementation

In [8]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to range [0, 1]
x_train = x_train / 255.0
x_test = x_test / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Build a simple feedforward neural network
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),             # Input shape matching MNIST image size
    tf.keras.layers.Flatten(),                         # Flatten 28x28 images to 1D vectors
    tf.keras.layers.Dense(64, activation='relu'),      # Hidden layer with 64 neurons and ReLU activation
    tf.keras.layers.Dense(10, activation='softmax')    # Output layer with 10 neurons (one per digit class)
])

# Compile the model with Adam optimizer and categorical crossentropy loss
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model and measure training time
start = time.time()
model.fit(x_train, y_train, epochs=5)
end = time.time()
print(f"TF Training time: {end-start:.2f} seconds")     # Print the training duration

# Evaluate the model on the test set
model.evaluate(x_test, y_test)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.8560 - loss: 0.5071
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9560 - loss: 0.1531
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9684 - loss: 0.1099
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9751 - loss: 0.0881
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9786 - loss: 0.0709
TF Training time: 22.74 seconds
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9677 - loss: 0.1044


[0.08921346068382263, 0.9728000164031982]

## Convert TensorFlow model to TFLite

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmpi671h43k'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor_15')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  138931313851472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138931313849936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138934006789136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138934006776848: TensorSpec(shape=(), dtype=tf.resource, name=None)


## PyTorch Implementation

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

# Transform: Convert to tensor and flatten 28x28 image to a 784-dimensional vector
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))  # Flatten the image
])

# Data loaders with different batch sizes
train_loader = DataLoader(
    datasets.MNIST(root='./data', train=True, transform=transform, download=True),
    batch_size=64,  # Changed batch size
    shuffle=True
)
test_loader = DataLoader(
    datasets.MNIST(root='./data', train=False, transform=transform, download=True),
    batch_size=512  # Changed batch size
)

# Define a deeper fully connected neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)    # More neurons in first layer
        self.fc2 = nn.Linear(256, 64)     # Added a new hidden layer
        self.fc3 = nn.Linear(64, 10)      # Output layer

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x))     # LeakyReLU instead of ReLU
        x = F.leaky_relu(self.fc2(x))
        return self.fc3(x)                # Final layer (logits)

# Instantiate model, loss function, and optimizer
model = Net()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.8)  # SGD instead of Adam

# Training loop
start = time.time()
for epoch in range(3):  # Fewer epochs
    for x, y in train_loader:
        optimizer.zero_grad()
        pred = model(x)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
end = time.time()
print(f"PyTorch Training time: {end - start:.2f} seconds")

# Evaluation loop
model.eval()
correct = 0
with torch.no_grad():
    for x, y in test_loader:
        output = model(x)
        pred = output.argmax(1)
        correct += (pred == y).sum().item()
print(f"Test accuracy: {correct / len(test_loader.dataset):.4f}")


PyTorch Training time: 25.99 seconds
Test accuracy: 0.9721


## Convert PyTorch model to ONNX

In [11]:
# Install ONNX
!pip install onnx

In [12]:
dummy_input = torch.randn(1, 784)
torch.onnx.export(model, dummy_input, "model.onnx",
                  input_names=["input"], output_names=["output"])

## TensorFlow custom training loop using tf.GradientTape

In [13]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Updated batch size
batch_size = 128

# Prepare datasets
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)) \
    .shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define a deeper model with different activations
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='tanh'),     # First hidden layer with tanh
    tf.keras.layers.Dense(64, activation='relu'),      # Second hidden layer with ReLU
    tf.keras.layers.Dense(10, activation='softmax')    # Output layer
])

# Set up training components with RMSprop optimizer
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

# Custom training loop
epochs = 3
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc_metric.update_state(y_batch, logits)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch + 1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Custom evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/3
Step 0, Loss: 2.2753, Accuracy: 0.1875
Step 100, Loss: 0.2774, Accuracy: 0.8443
Step 200, Loss: 0.2559, Accuracy: 0.8791
Step 300, Loss: 0.1419, Accuracy: 0.8955
Step 400, Loss: 0.2378, Accuracy: 0.9055
Training Accuracy for epoch 1: 0.9117

Epoch 2/3
Step 0, Loss: 0.2334, Accuracy: 0.9141
Step 100, Loss: 0.2442, Accuracy: 0.9518
Step 200, Loss: 0.1778, Accuracy: 0.9545
Step 300, Loss: 0.1513, Accuracy: 0.9561
Step 400, Loss: 0.1603, Accuracy: 0.9566
Training Accuracy for epoch 2: 0.9578

Epoch 3/3
Step 0, Loss: 0.1161, Accuracy: 0.9922
Step 100, Loss: 0.1103, Accuracy: 0.9678
Step 200, Loss: 0.1525, Accuracy: 0.9682
Step 300, Loss: 0.1562, Accuracy: 0.9691
Step 400, Loss: 0.1067, Accuracy: 0.9687
Training Accuracy for epoch 3: 0.9694

TF Training time: 58.58 seconds
Test Accuracy: 0.9707


## Performance Otimization with Graph Execution using @tf.function

In [14]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Updated batch size
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define a deeper model with LeakyReLU and ReLU
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256),
    tf.keras.layers.LeakyReLU(negative_slope=0.1),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Updated optimizer to SGD
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

@tf.function
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        logits = model(x_batch, training=True)
        loss = loss_fn(y_batch, logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_acc_metric.update_state(y_batch, logits)
    return loss

# Training loop
epochs = 4
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        loss = train_step(x_batch, y_batch)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch + 1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/4
Step 0, Loss: 2.3849, Accuracy: 0.0156
Step 100, Loss: 1.6642, Accuracy: 0.4675
Step 200, Loss: 1.0007, Accuracy: 0.6066
Step 300, Loss: 0.6345, Accuracy: 0.6771
Step 400, Loss: 0.6329, Accuracy: 0.7161
Step 500, Loss: 0.7429, Accuracy: 0.7444
Step 600, Loss: 0.5555, Accuracy: 0.7643
Step 700, Loss: 0.4845, Accuracy: 0.7796
Step 800, Loss: 0.3205, Accuracy: 0.7927
Step 900, Loss: 0.3830, Accuracy: 0.8043
Training Accuracy for epoch 1: 0.8082

Epoch 2/4
Step 0, Loss: 0.3873, Accuracy: 0.9062
Step 100, Loss: 0.4358, Accuracy: 0.8957
Step 200, Loss: 0.2541, Accuracy: 0.8965
Step 300, Loss: 0.3790, Accuracy: 0.8957
Step 400, Loss: 0.3437, Accuracy: 0.8957
Step 500, Loss: 0.4167, Accuracy: 0.8961
Step 600, Loss: 0.5236, Accuracy: 0.8965
Step 700, Loss: 0.2462, Accuracy: 0.8965
Step 800, Loss: 0.2277, Accuracy: 0.8984
Step 900, Loss: 0.4395, Accuracy: 0.9006
Training Accuracy for epoch 2: 0.9010

Epoch 3/4
Step 0, Loss: 0.1471, Accuracy: 0.9375
Step 100, Loss: 0.3394, Accuracy: 0.